In [1]:
import numpy as np
import pandas as pd
import json
import difflib
from fuzzywuzzy import fuzz
pd.options.display.max_rows = 100

####Function to fix names

In [2]:
"""
Functions
--------
fix_names recieves a dataframe(containing transfer data) where it cleans
the dataframe 'name' column.

Inputs
------
df: a dataframe which contains transfer value with the following details.
currency: currency of transfer amount
date: date of the transfer
from: Former club of the player
name: Name of the player
to: New club of the player
value: Transfer value/type

Returns
-------
A dataframe where the names have been fixed for further processing.

Notes
-----
fix_name is a helper function for the fix_names function.
Written 28 Nov. 2015 by ARM
"""
def fix_name(name_list):
    if len(name_list) == 1:
        return name_list[0]
    elif len(name_list) == 2:
        val = ' '.join([name_list[1],name_list[0]])
        return val.lstrip().rstrip()
    else:
        return 'Name Error'

def fix_names(df):
    df['name'] = df['name'].apply(lambda x:x.split(','))
    df['name'] = df['name'].apply(lambda x:fix_name(x))
    df['name'] = df['name'].astype(unicode)
    df = df.set_index('name')
    return df

####Function to clean and match names and join the values

In [3]:
"""
Functions
--------
fix_names recieves a dataframe(containing transfer data) where it cleans
the dataframe 'name' column.

Inputs
------
transfer_df: a dataframe which contains transfer data cleaned with
fix_names function.
stats_df: a dataframe which contains the feature data collected for the
corresponding season.
Returns
-------
A joined dataframe which collects the common players in the data.

Notes
-----
fix_name is a helper function for the fix_names function.
Written 28 Nov. 2015 by ARM
"""
def join_df(transfer_df,stats_df):
    test_df = transfer_df
    test_df['check_name'] = test_df.index.map(lambda x: difflib.get_close_matches(x, stats_df.index,1))
    test_df = test_df[test_df['check_name'].str.len() != 0]
    test_df['check_name'] = test_df['check_name'].apply(lambda x:x[0])
    test_df['names'] = test_df.index
    test_df['sim_score'] = test_df[['check_name','names']].apply(lambda x:fuzz.ratio(x[0],x[1]),axis=1)
    to_be_joined = test_df[test_df['sim_score'] > 80]
    to_be_joined['ref_name'] = to_be_joined['check_name']
    to_be_joined = to_be_joined.set_index('check_name')
    trial_df = stats_df.join(to_be_joined,how='inner')
    return trial_df

#### Loading feature data for season 2012-13

In [4]:
features12 = pd.read_pickle('../Data/features12-13.pkl')
tr_features12 = features12.transpose()

####Loading feature data for season 2013-14

In [5]:
features13 = pd.read_pickle('../Data/features13-14.pkl')
tr_features13 = features13.transpose()

#### Loading transfer data for 2012-13 for England

In [6]:
transfers_england_summer = pd.read_json('../Transfer_data/England_summer_2013.json')
transfers_england_winter = pd.read_json('../Transfer_data/England_winter_2012_13.json')
transfers_england_summer['value'] = transfers_england_summer['value'].astype(str)
transfers_england_winter['value'] = transfers_england_winter['value'].astype(str)

#Concatenating all transfers for the season 12-13 for England
df_eng_12_13 = pd.concat([transfers_england_summer,transfers_england_winter],axis=0)

####Processing data for 2012-13 for England

In [ ]:
england_2012_13_df = join_df(fix_names(df_eng_12_13),tr_features12)

In [8]:
#Deleting columns with lists before saving
england_2012_13_df = england_2012_13_df.drop(['smoothed_value_list',\
                    'match_value_list','cum_value_list'],axis=1)
#Resetting the dataframe
england_2012_13_df = england_2012_13_df.reset_index()

####Loading data for Spain for 2012-13

In [9]:
transfers_spain_summer = pd.read_json('../Transfer_data/Spain_summer_2013.json')
transfers_spain_winter = pd.read_json('../Transfer_data/Spain_winter_2012_13.json')
transfers_spain_summer['value'] = transfers_spain_summer['value'].astype(str)
transfers_spain_winter['value'] = transfers_spain_winter['value'].astype(str)

#Concatenating all transfers for the season 12-13 for Spain
df_spn_12_13 = pd.concat([transfers_spain_summer,transfers_spain_winter],axis=0)

####Processing data for 2012-13 for Spain

In [ ]:
spain_2012_13_df = join_df(fix_names(df_spn_12_13),tr_features12)

In [11]:
#Deleting columns with lists before saving
spain_2012_13_df = spain_2012_13_df.drop(['smoothed_value_list',\
                    'match_value_list','cum_value_list'],axis=1)
#Resetting the dataframe
spain_2012_13_df = spain_2012_13_df.reset_index()

####Loading data for Italy for 2012-13

In [12]:
transfers_italy_summer_jul = pd.read_json('../Transfer_data/Italy_summer_2013_Jul.json')
transfers_italy_summer_aug = pd.read_json('../Transfer_data/Italy_summer_2013_Aug.json')
transfers_italy_winter = pd.read_json('../Transfer_data/Spain_winter_2012_13.json')

#Concatenating all transfers for the season 12-13 for Italy
df_it_12_13 = pd.concat([transfers_italy_summer_jul,\
                         transfers_italy_summer_aug, transfers_italy_winter],axis=0)

####Processing data for 2012-13 for Italy

In [ ]:
italy_2012_13_df = join_df(fix_names(df_it_12_13),tr_features12)

In [14]:
#Deleting columns with lists before saving
italy_2012_13_df = italy_2012_13_df.drop(['smoothed_value_list',\
                    'match_value_list','cum_value_list'],axis=1)
#Resetting the dataframe
italy_2012_13_df = italy_2012_13_df.reset_index()

####Loading data for 2013-14 for England

In [15]:
transfers_england_summer = pd.read_json('../Transfer_data/England_summer_2014.json')
transfers_england_winter = pd.read_json('../Transfer_data/England_winter_2013_14.json')
transfers_england_summer['value'] = transfers_england_summer['value'].astype(str)
transfers_england_winter['value'] = transfers_england_winter['value'].astype(str)

#Concatenating all transfers for the season 13-14 for England
df_eng_13_14 = pd.concat([transfers_england_summer,transfers_england_winter],axis=0)

####Processing data for 2013-14 for England

In [ ]:
england_2013_14_df = join_df(fix_names(df_eng_13_14),tr_features13)

In [34]:
#Deleting columns with lists before saving
england_2013_14_df = england_2013_14_df.drop(['smoothed_value_list',\
                    'match_value_list','cum_value_list'],axis=1)
#Resetting the dataframe
england_2013_14_df = england_2013_14_df.reset_index()

####Saving only england data

In [42]:
val_eng_12_13 = england_2012_13_df[england_2012_13_df['value'].apply(lambda x: x.isdigit())]
val_eng_13_14 = england_2013_14_df[england_2013_14_df['value'].apply(lambda x: x.isdigit())]
val_eng = pd.concat([val_eng_12_13,val_eng_13_14],axis=0)
val_eng = val_eng.reset_index()
val_eng = val_eng.drop(['level_0','index','ref_name'], axis=1)
val_eng.to_excel("England_all_transfers_with_value.xlsx")

In [43]:
val_eng_12_13 = england_2012_13_df[~england_2012_13_df['value'].apply(lambda x: x.isdigit())]
val_eng_13_14 = england_2013_14_df[~england_2013_14_df['value'].apply(lambda x: x.isdigit())]
val_eng = pd.concat([val_eng_12_13,val_eng_13_14],axis=0)
val_eng = val_eng.reset_index()
val_eng = val_eng.drop(['level_0','index','ref_name'], axis=1)
val_eng.to_excel("England_all_transfers_without_value.xlsx")

In [17]:
check_list = ['Loan','Undisclosed','Swap','Free','Loan return']
#test_df = df_eng_12_13[~df_eng_12_13['value'].isin(check_list)]

In [18]:
##Testing the amount of transfer money we can account for after name fix
sum(spain_2012_13_df[~spain_2012_13_df['value'].isin(check_list)]['value'].astype(float))

262000000.0